In [119]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

c:\Users\Nived Shah\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [120]:
device = torch.device('cuda:0')
model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True).to(device)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\Nived Shah\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nived Shah\.cache\huggingface\hub\models--Alibaba-NLP--gte-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# a list of reference section headings in a resume (case-insensitive)
reference_section_headings = [
    "Achievements",
    "Affiliations",
    "Awards and Honors",
    "Background",
    "Certifications",
    "Contact Information",
    "Education",
    "Experience",
    "Extracurricular Activities",
    "Hobbies and Interests",
    "Internships",
    "Languages",
    "Personal Information",
    "Professional Experience",
    "Projects",
    "Publications",
    "Research",
    "Skills",
    "Summary",
    "Training",
    "Volunteering",
]

same_meanig_words = {
    "Summary": ["Obejective", "Statement", "Career", "Goal", "Profile"]
}

embeddings = model.encode(reference_section_headings, convert_to_tensor=True, show_progress_bar = False)
cosine_scores = cos_sim(embeddings[0], embeddings[1]).cpu().numpy()[0][0]

In [114]:
# identify resume sections by finding the first capital letter in each line
def is_section(line):
    if len(line) > 0 and len(list(filter((lambda x: x != ''), line.split(" ")))) <= 3:
        if line[0].isupper():
            for ref in reference_section_headings:
                check_section_closeness = [ref, line]
                embeddings = model.encode(check_section_closeness, convert_to_tensor=True, show_progress_bar = False)
                cosine_scores = cos_sim(embeddings[0], embeddings[1]).cpu().numpy()[0][0]
                if cosine_scores > 0.8:
                    return ref
                if ref in same_meanig_words:
                    for syn in same_meanig_words[ref]:
                        check_section_closeness = [syn, line]
                        embeddings = model.encode(check_section_closeness, convert_to_tensor=True, show_progress_bar = False)
                        cosine_scores = cos_sim(embeddings[0], embeddings[1]).cpu().numpy()[0][0]
                        if cosine_scores > 0.8:
                            return ref
    return None
            

# assumption: the section heading is followed by a newline

In [115]:
# Function to extract sections and content from the full resume text
def extract_sections_from_text(resume_text):

    sections = {"skiplines": []}
    current_section = "skiplines"

    # Split the text into lines
    lines = resume_text.splitlines()

    for line in lines:
        line = line.strip()
        section = is_section(line)
        if section:
            current_section = section
            sections[current_section] = []
        else:
            # concat the current line with the last list element in the current section content list
            if len(sections[current_section]) > 0:
                sections[current_section][-1] = sections[current_section][-1] + '\n' + line
            else:
                sections[current_section].append(line)

            

    return sections


In [116]:
def process_resumes(csv_file):
    # Load the CSV into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Create a list to hold the extracted sections and content
    extracted_data = []

    # Iterate over each resume
    for index, row in df.iterrows():
        resume_id = row['id']
        job_title = row['title']
        full_text = row['text'] 
        
        # Extract sections from the full text of the resume
        sections = extract_sections_from_text(full_text)

        # Store each section with the resume ID and job title
        for section in sections:
            extracted_data.append({
                'Resume ID': resume_id,
                'Job Title': job_title,
                'Section': section,
                'Content': sections[section]
            })

    # Convert the extracted data into a DataFrame
    extracted_df = pd.DataFrame(extracted_data)
    return extracted_df

In [117]:
csv_file_path = "../Final_Resume_Texts/resume_texts.csv"

# df = pd.read_csv(csv_file_path)

# print(df.head())

# Process the resumes and get the extracted sections DataFrame
extracted_df = process_resumes(csv_file_path)

# # Display the extracted DataFrame
print(extracted_df)

# # Optionally, save to a CSV file
extracted_df.to_csv('extracted_resume_sections.csv', index=False)

NameError: name 'reference_section_headings' is not defined